In [8]:
%load_ext autoreload
%autoreload 2

from train import *
from models import *
from tools import load_data, get_label_dict, get_word_labels

from keras import optimizers
from keras.models import load_model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

from PIL import Image
from IPython.display import display

import numpy as np
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
X_train, y_train = load_data()
X_val, y_val = load_data("val")
X_test, _ = load_data("test")

y_train = to_categorical(y_train, 200)
y_val = to_categorical(y_val, 200)

Loading train data

Finished loading train data

Loading val data

Finished loading val data

Loading test data

Finished loading test data



In [10]:
labels = get_label_dict()
words = get_word_labels()

test_images = os.listdir("data/tiny-imagenet-200/test/images/")
assert len(X_test) == len(test_images)

Loading labels

Done

Loading words

Done



In [11]:
# Image preprocessing for the data
datagen = ImageDataGenerator(rotation_range=40, 
                             brightness_range=[0.6, 1.4], 
                             shear_range=0.2, 
                             zoom_range=0.2, 
                             horizontal_flip=True, 
                             vertical_flip=True, 
                             width_shift_range=0.2,
                             height_shift_range=0.2, 
                             channel_shift_range=90., 
                             fill_mode="reflect", 
                             preprocessing_function=shuffle_channels, 
                             rescale=1./255)

testgen = ImageDataGenerator(rescale=1./255)

data = {"train": (X_train, y_train), 
        "val": (X_val, y_val)}

In [12]:
model = AlphaNet()
model.name = "AlphaNet_v2"
model_prefix = "models/" + model.name + "/" + model.name
model_path = model_prefix + ".h5"

In [13]:
# Checkpoint callback to only save best model
checkpoint = ModelCheckpoint(model_path, 
                             monitor="val_accuracy", 
                             verbose=1, 
                             save_best_only=True, 
                             mode="auto", 
                             period=1)

cb_list = [checkpoint]

In [14]:
optim = optimizers.Adam(learning_rate=0.001)

history = train(model_path=model_path, 
                restore=False, 
                epochs=300,
                model=model, 
                optim=optim,
                datagen=datagen,
                data=data, 
                cb_list=cb_list)

Model: "AlphaNet_v2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 64, 64, 32)        4736      
_________________________________________________________________
batch_normalization_8 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
spatial_dropout2d_5 (Spatial (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        51264     
_________________________________________________________________
batch_normalization_9 (Batch (None, 32, 32, 64)        


Epoch 00010: val_accuracy did not improve from 0.05790
Epoch 11/300
3125/3125 [==============================] - 186s 59ms/step - loss: 4.7690 - accuracy: 0.0485 - top3_accuracy: 0.1127 - top5_accuracy: 0.1626 - val_loss: 4.6336 - val_accuracy: 0.0522 - val_top3_accuracy: 0.1241 - val_top5_accuracy: 0.1749

Epoch 00011: val_accuracy did not improve from 0.05790
Epoch 12/300
3125/3125 [==============================] - 185s 59ms/step - loss: 4.7411 - accuracy: 0.0519 - top3_accuracy: 0.1187 - top5_accuracy: 0.1700 - val_loss: 4.8033 - val_accuracy: 0.0627 - val_top3_accuracy: 0.1385 - val_top5_accuracy: 0.1971

Epoch 00012: val_accuracy improved from 0.05790 to 0.06270, saving model to models/AlphaNet_v2/AlphaNet_v2.h5
Epoch 13/300
3125/3125 [==============================] - 185s 59ms/step - loss: 4.7150 - accuracy: 0.0552 - top3_accuracy: 0.1248 - top5_accuracy: 0.1771 - val_loss: 5.0166 - val_accuracy: 0.0709 - val_top3_accuracy: 0.1534 - val_top5_accuracy: 0.2172

Epoch 00013: val_


Epoch 00035: val_accuracy did not improve from 0.12100
Epoch 36/300
3125/3125 [==============================] - 186s 59ms/step - loss: 4.3865 - accuracy: 0.0940 - top3_accuracy: 0.1929 - top5_accuracy: 0.2595 - val_loss: 3.6747 - val_accuracy: 0.1167 - val_top3_accuracy: 0.2348 - val_top5_accuracy: 0.3104

Epoch 00036: val_accuracy did not improve from 0.12100
Epoch 37/300
3125/3125 [==============================] - 185s 59ms/step - loss: 4.3792 - accuracy: 0.0935 - top3_accuracy: 0.1925 - top5_accuracy: 0.2597 - val_loss: 3.7443 - val_accuracy: 0.1195 - val_top3_accuracy: 0.2346 - val_top5_accuracy: 0.3115

Epoch 00037: val_accuracy did not improve from 0.12100
Epoch 38/300
3125/3125 [==============================] - 185s 59ms/step - loss: 4.3696 - accuracy: 0.0945 - top3_accuracy: 0.1939 - top5_accuracy: 0.2627 - val_loss: 3.4354 - val_accuracy: 0.1252 - val_top3_accuracy: 0.2500 - val_top5_accuracy: 0.3292

Epoch 00038: val_accuracy improved from 0.12100 to 0.12520, saving model


Epoch 00060: val_accuracy did not improve from 0.15590
Epoch 61/300
3125/3125 [==============================] - 185s 59ms/step - loss: 4.2145 - accuracy: 0.1151 - top3_accuracy: 0.2288 - top5_accuracy: 0.3004 - val_loss: 4.3191 - val_accuracy: 0.1448 - val_top3_accuracy: 0.2743 - val_top5_accuracy: 0.3517

Epoch 00061: val_accuracy did not improve from 0.15590
Epoch 62/300
3125/3125 [==============================] - 185s 59ms/step - loss: 4.2114 - accuracy: 0.1155 - top3_accuracy: 0.2307 - top5_accuracy: 0.3034 - val_loss: 3.5692 - val_accuracy: 0.1547 - val_top3_accuracy: 0.2862 - val_top5_accuracy: 0.3656

Epoch 00062: val_accuracy did not improve from 0.15590
Epoch 63/300
3125/3125 [==============================] - 185s 59ms/step - loss: 4.2049 - accuracy: 0.1165 - top3_accuracy: 0.2304 - top5_accuracy: 0.3032 - val_loss: 4.3177 - val_accuracy: 0.1445 - val_top3_accuracy: 0.2770 - val_top5_accuracy: 0.3579

Epoch 00063: val_accuracy did not improve from 0.15590
Epoch 64/300
3125

Epoch 86/300
3125/3125 [==============================] - 186s 59ms/step - loss: 4.1089 - accuracy: 0.1312 - top3_accuracy: 0.2522 - top5_accuracy: 0.3269 - val_loss: 3.9190 - val_accuracy: 0.1809 - val_top3_accuracy: 0.3308 - val_top5_accuracy: 0.4169

Epoch 00086: val_accuracy did not improve from 0.18190
Epoch 87/300
3125/3125 [==============================] - 185s 59ms/step - loss: 4.0998 - accuracy: 0.1304 - top3_accuracy: 0.2535 - top5_accuracy: 0.3303 - val_loss: 3.7944 - val_accuracy: 0.1823 - val_top3_accuracy: 0.3262 - val_top5_accuracy: 0.4076

Epoch 00087: val_accuracy improved from 0.18190 to 0.18230, saving model to models/AlphaNet_v2/AlphaNet_v2.h5
Epoch 88/300
3125/3125 [==============================] - 185s 59ms/step - loss: 4.0974 - accuracy: 0.1320 - top3_accuracy: 0.2533 - top5_accuracy: 0.3282 - val_loss: 3.6590 - val_accuracy: 0.1809 - val_top3_accuracy: 0.3295 - val_top5_accuracy: 0.4139

Epoch 00088: val_accuracy did not improve from 0.18230
Epoch 89/300
3125/

3125/3125 [==============================] - 186s 59ms/step - loss: 4.0205 - accuracy: 0.1438 - top3_accuracy: 0.2698 - top5_accuracy: 0.3474 - val_loss: 3.1284 - val_accuracy: 0.1962 - val_top3_accuracy: 0.3447 - val_top5_accuracy: 0.4315

Epoch 00111: val_accuracy did not improve from 0.19910
Epoch 112/300
3125/3125 [==============================] - 186s 59ms/step - loss: 4.0172 - accuracy: 0.1440 - top3_accuracy: 0.2711 - top5_accuracy: 0.3489 - val_loss: 3.9788 - val_accuracy: 0.2004 - val_top3_accuracy: 0.3532 - val_top5_accuracy: 0.4374

Epoch 00112: val_accuracy improved from 0.19910 to 0.20040, saving model to models/AlphaNet_v2/AlphaNet_v2.h5
Epoch 113/300
3125/3125 [==============================] - 185s 59ms/step - loss: 4.0165 - accuracy: 0.1449 - top3_accuracy: 0.2722 - top5_accuracy: 0.3492 - val_loss: 3.3781 - val_accuracy: 0.1972 - val_top3_accuracy: 0.3550 - val_top5_accuracy: 0.4413

Epoch 00113: val_accuracy did not improve from 0.20040
Epoch 114/300
3125/3125 [====

3125/3125 [==============================] - 185s 59ms/step - loss: 3.9565 - accuracy: 0.1524 - top3_accuracy: 0.2833 - top5_accuracy: 0.3631 - val_loss: 3.1805 - val_accuracy: 0.2046 - val_top3_accuracy: 0.3619 - val_top5_accuracy: 0.4474

Epoch 00136: val_accuracy did not improve from 0.21760
Epoch 137/300
3125/3125 [==============================] - 185s 59ms/step - loss: 3.9519 - accuracy: 0.1513 - top3_accuracy: 0.2850 - top5_accuracy: 0.3646 - val_loss: 3.3969 - val_accuracy: 0.2199 - val_top3_accuracy: 0.3789 - val_top5_accuracy: 0.4655

Epoch 00137: val_accuracy improved from 0.21760 to 0.21990, saving model to models/AlphaNet_v2/AlphaNet_v2.h5
Epoch 138/300
3125/3125 [==============================] - 185s 59ms/step - loss: 3.9506 - accuracy: 0.1547 - top3_accuracy: 0.2863 - top5_accuracy: 0.3656 - val_loss: 3.8272 - val_accuracy: 0.2093 - val_top3_accuracy: 0.3676 - val_top5_accuracy: 0.4493

Epoch 00138: val_accuracy did not improve from 0.21990
Epoch 139/300
3125/3125 [====


Epoch 00161: val_accuracy did not improve from 0.22430
Epoch 162/300
3125/3125 [==============================] - 186s 60ms/step - loss: 3.9036 - accuracy: 0.1613 - top3_accuracy: 0.2968 - top5_accuracy: 0.3768 - val_loss: 3.3817 - val_accuracy: 0.2227 - val_top3_accuracy: 0.3794 - val_top5_accuracy: 0.4671

Epoch 00162: val_accuracy did not improve from 0.22430
Epoch 163/300
3125/3125 [==============================] - 185s 59ms/step - loss: 3.8998 - accuracy: 0.1605 - top3_accuracy: 0.2965 - top5_accuracy: 0.3773 - val_loss: 3.6570 - val_accuracy: 0.2289 - val_top3_accuracy: 0.3888 - val_top5_accuracy: 0.4753

Epoch 00163: val_accuracy improved from 0.22430 to 0.22890, saving model to models/AlphaNet_v2/AlphaNet_v2.h5
Epoch 164/300
3125/3125 [==============================] - 185s 59ms/step - loss: 3.8948 - accuracy: 0.1615 - top3_accuracy: 0.2972 - top5_accuracy: 0.3771 - val_loss: 3.3263 - val_accuracy: 0.2239 - val_top3_accuracy: 0.3877 - val_top5_accuracy: 0.4715

Epoch 00164: v


Epoch 00187: val_accuracy did not improve from 0.23520
Epoch 188/300
3125/3125 [==============================] - 185s 59ms/step - loss: 3.8670 - accuracy: 0.1670 - top3_accuracy: 0.3039 - top5_accuracy: 0.3841 - val_loss: 3.5715 - val_accuracy: 0.2293 - val_top3_accuracy: 0.3965 - val_top5_accuracy: 0.4815

Epoch 00188: val_accuracy did not improve from 0.23520
Epoch 189/300
3125/3125 [==============================] - 185s 59ms/step - loss: 3.8567 - accuracy: 0.1690 - top3_accuracy: 0.3073 - top5_accuracy: 0.3885 - val_loss: 3.7100 - val_accuracy: 0.2338 - val_top3_accuracy: 0.3984 - val_top5_accuracy: 0.4863

Epoch 00189: val_accuracy did not improve from 0.23520
Epoch 190/300
3125/3125 [==============================] - 185s 59ms/step - loss: 3.8548 - accuracy: 0.1692 - top3_accuracy: 0.3066 - top5_accuracy: 0.3871 - val_loss: 3.3596 - val_accuracy: 0.2261 - val_top3_accuracy: 0.3906 - val_top5_accuracy: 0.4801

Epoch 00190: val_accuracy did not improve from 0.23520
Epoch 191/300


KeyboardInterrupt: 

In [ ]:
show_history(history)

In [ ]:
# This loads the model and makes predictions on the test set
custom_metrics = {
    "top3_accuracy": top3_acc,
    "top5_accuracy": top5_acc
}
model = load_model(model_path, custom_objects=custom_metrics)
output = model.predict_generator(testgen.flow(X_test, shuffle=False))
results = []

for result in output:
    results.append(labels[np.argmax(result)])
    
assert len(results) == len(X_test)
print("Test images predicted:", len(results))

In [ ]:
# This writes the test set predictions to file
with open(model_prefix + " Predictions.txt", "w") as test_file:
    for i in range(len(results)):
        test_file.write(test_images[i] + " " + results[i] + "\n")
    print("Predictions saved at", test_file.name)

In [ ]:
# This shows some test set images and the model's predictions on them
display_dim = 128
for _ in range(10):
    index = np.random.randint(len(X_test))
    img = Image.fromarray(X_test[index], "RGB").resize(size=(display_dim, display_dim))
    word_label = words[results[index]]
    display(img)
    print(word_label)

In [ ]:
# This saves an HTML version of the notebook for later viewing
os.system("%notebook 'Training'")
os.system("jupyter nbconvert --to html Training.ipynb")

html_path = model_prefix + ".html"
if path.exists(html_path):
    os.remove(html_path)
os.rename("Training.html", html_path)